# 0) Aufsetzen der Umgebung

In [1]:
!pip install gym_chess
!pip install cairosvg
!pip install opencv-python
!mkdir ./videos

mkdir: cannot create directory ‘./videos’: File exists


In [2]:
import numpy as np
import random
import gym
import gym_chess
import chess
import chess.svg
import chess.polyglot

from datetime import datetime
import timeit
from io import BytesIO
from PIL import Image
from cairosvg import svg2png
import cv2
from IPython.display import clear_output

In [3]:
def write_video(svgs):
  out = cv2.VideoWriter(f'videos/game_{datetime.now()}.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 3, (350, 350))

  for svg in svgs:
    png = convert_svg_to_png(svg)
    out.write(png)

  out.release()


def convert_svg_to_png(svg):  
  png = svg2png(svg)
  pil_img = Image.open(BytesIO(png)).convert('RGBA')
  png = cv2.cvtColor(np.array(pil_img), cv2.IMREAD_COLOR)  

  return png

# 1) Minimax implementation

https://andreasstckl.medium.com/writing-a-chess-program-in-one-day-30daff4610ec

In [4]:
def evaluate_board():
    
    if board.is_checkmate():
        if board.turn:
            return -9999
        else:
            return 9999
    if board.is_stalemate():
        return 0
    if board.is_insufficient_material():
        return 0
    
    wp = len(board.pieces(chess.PAWN, chess.WHITE))
    bp = len(board.pieces(chess.PAWN, chess.BLACK))
    wn = len(board.pieces(chess.KNIGHT, chess.WHITE))
    bn = len(board.pieces(chess.KNIGHT, chess.BLACK))
    wb = len(board.pieces(chess.BISHOP, chess.WHITE))
    bb = len(board.pieces(chess.BISHOP, chess.BLACK))
    wr = len(board.pieces(chess.ROOK, chess.WHITE))
    br = len(board.pieces(chess.ROOK, chess.BLACK))
    wq = len(board.pieces(chess.QUEEN, chess.WHITE))
    bq = len(board.pieces(chess.QUEEN, chess.BLACK))
    
    material = 100*(wp-bp)+320*(wn-bn)+330*(wb-bb)+500*(wr-br)+900*(wq-bq)
    
    pawnsq = sum([pawntable[i] for i in board.pieces(chess.PAWN, chess.WHITE)])
    pawnsq= pawnsq + sum([-pawntable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.PAWN, chess.BLACK)])
    knightsq = sum([knightstable[i] for i in board.pieces(chess.KNIGHT, chess.WHITE)])
    knightsq = knightsq + sum([-knightstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.KNIGHT, chess.BLACK)])
    bishopsq= sum([bishopstable[i] for i in board.pieces(chess.BISHOP, chess.WHITE)])
    bishopsq= bishopsq + sum([-bishopstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.BISHOP, chess.BLACK)])
    rooksq = sum([rookstable[i] for i in board.pieces(chess.ROOK, chess.WHITE)]) 
    rooksq = rooksq + sum([-rookstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.ROOK, chess.BLACK)])
    queensq = sum([queenstable[i] for i in board.pieces(chess.QUEEN, chess.WHITE)]) 
    queensq = queensq + sum([-queenstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.QUEEN, chess.BLACK)])
    kingsq = sum([kingstable[i] for i in board.pieces(chess.KING, chess.WHITE)]) 
    kingsq = kingsq + sum([-kingstable[chess.square_mirror(i)] 
                                    for i in board.pieces(chess.KING, chess.BLACK)])
    
    eval = material + pawnsq + knightsq + bishopsq+ rooksq+ queensq + kingsq
    if board.turn:
        return eval
    else:
        return -eval

In [5]:
pawntable = [
 0,  0,  0,  0,  0,  0,  0,  0,
 5, 10, 10,-20,-20, 10, 10,  5,
 5, -5,-10,  0,  0,-10, -5,  5,
 0,  0,  0, 20, 20,  0,  0,  0,
 5,  5, 10, 25, 25, 10,  5,  5,
10, 10, 20, 30, 30, 20, 10, 10,
50, 50, 50, 50, 50, 50, 50, 50,
 0,  0,  0,  0,  0,  0,  0,  0]

knightstable = [
-50,-40,-30,-30,-30,-30,-40,-50,
-40,-20,  0,  5,  5,  0,-20,-40,
-30,  5, 10, 15, 15, 10,  5,-30,
-30,  0, 15, 20, 20, 15,  0,-30,
-30,  5, 15, 20, 20, 15,  5,-30,
-30,  0, 10, 15, 15, 10,  0,-30,
-40,-20,  0,  0,  0,  0,-20,-40,
-50,-40,-30,-30,-30,-30,-40,-50]

bishopstable = [
-20,-10,-10,-10,-10,-10,-10,-20,
-10,  5,  0,  0,  0,  0,  5,-10,
-10, 10, 10, 10, 10, 10, 10,-10,
-10,  0, 10, 10, 10, 10,  0,-10,
-10,  5,  5, 10, 10,  5,  5,-10,
-10,  0,  5, 10, 10,  5,  0,-10,
-10,  0,  0,  0,  0,  0,  0,-10,
-20,-10,-10,-10,-10,-10,-10,-20]

rookstable = [
  0,  0,  0,  5,  5,  0,  0,  0,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
 -5,  0,  0,  0,  0,  0,  0, -5,
  5, 10, 10, 10, 10, 10, 10,  5,
 0,  0,  0,  0,  0,  0,  0,  0]

queenstable = [
-20,-10,-10, -5, -5,-10,-10,-20,
-10,  0,  0,  0,  0,  0,  0,-10,
-10,  5,  5,  5,  5,  5,  0,-10,
  0,  0,  5,  5,  5,  5,  0, -5,
 -5,  0,  5,  5,  5,  5,  0, -5,
-10,  0,  5,  5,  5,  5,  0,-10,
-10,  0,  0,  0,  0,  0,  0,-10,
-20,-10,-10, -5, -5,-10,-10,-20]

kingstable = [
 20, 30, 10,  0,  0, 10, 30, 20,
 20, 20,  0,  0,  0,  0, 20, 20,
-10,-20,-20,-20,-20,-20,-20,-10,
-20,-30,-30,-40,-40,-30,-30,-20,
-30,-40,-40,-50,-50,-40,-40,-30,
-30,-40,-40,-50,-50,-40,-40,-30,
-30,-40,-40,-50,-50,-40,-40,-30,
-30,-40,-40,-50,-50,-40,-40,-30]

In [6]:
def alphabeta( alpha, beta, depthleft ):
    bestscore = -9999
    if( depthleft == 0 ):
        return quiesce( alpha, beta )
    for move in board.legal_moves:
        board.push(move)   
        score = -alphabeta( -beta, -alpha, depthleft - 1 )
        board.pop()
        if( score >= beta ):
            return score
        if( score > bestscore ):
            bestscore = score
        if( score > alpha ):
            alpha = score   
    return bestscore

In [7]:

def quiesce( alpha, beta ):
    stand_pat = evaluate_board()
    if( stand_pat >= beta ):
        return beta
    if( alpha < stand_pat ):
        alpha = stand_pat

    for move in board.legal_moves:
        if board.is_capture(move):
            board.push(move)        
            score = -quiesce( -beta, -alpha )
            board.pop()

            if( score >= beta ):
                return beta
            if( score > alpha ):
                alpha = score  
    return alpha

In [8]:
def selectmove(depth):
    try:
        move = chess.polyglot.MemoryMappedReader("bookfish.bin").weighted_choice(board).move()
        return move
    except:
        bestMove = chess.Move.null()
        bestValue = -99999
        alpha = -100000
        beta = 100000
        for move in board.legal_moves:
            board.push(move)
            boardValue = -alphabeta(-beta, -alpha, depth-1)
            if boardValue > bestValue:
                bestValue = boardValue;
                bestMove = move
            if( boardValue > alpha ):
                alpha = boardValue
            board.pop()
        return bestMove

# 2) Minimax vs Random Agent

In [9]:
env = gym.make('Chess-v0')
observation = env.reset()
board = observation

# variables
total_reward = 0.0
total_steps = 0
moves = []
svgs = []

while True:
  start = timeit.default_timer()
  action = selectmove(2)

  clear_output(wait=True)
  print(f'Action: {action} in {(timeit.default_timer()-start):.3f} sec')
  print(env.render(mode='unicode'))
  
  observation, reward, done, info = env.step(action) 
  board = observation

  moves.append(action)
  total_reward += reward
  total_steps += 1

  svgs.append(chess.svg.board(observation, size=350))
    
  if done: 
    clear_output(wait=True)
    print(f'Action: {action} in {(timeit.default_timer()-start):.3f} sec')
    print(env.render(mode='unicode'))
    break;
            
env.close()
write_video(svgs)

result = ""
if(total_reward == 0):
  result = "draw"
if(total_reward == 1):
  result = "won"
if(total_reward == -1):
  result = "lost"

print(f"{result} in {total_steps} moves: {moves}")

Action: f5e6 in 0.080 sec
⭘ ⭘ ⭘ ♜ ⭘ ♝ ♚ ♜
⭘ ♟ ♟ ⭘ ♟ ⭘ ♟ ♟
⭘ ⭘ ⭘ ⭘ ♕ ⭘ ⭘ ⭘
⭘ ♛ ⭘ ♟ ♙ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ♗ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ♙ ⭘ ⭘ ♗ ⭘ ⭘
⭘ ⭘ ♙ ⭘ ⭘ ♙ ♙ ♙
♖ ⭘ ⭘ ⭘ ♖ ⭘ ♔ ⭘
won in 43 moves: [Move.from_uci('g1f3'), Move.from_uci('g8f6'), Move.from_uci('b1c3'), Move.from_uci('b8c6'), Move.from_uci('e2e4'), Move.from_uci('d7d5'), Move.from_uci('e4e5'), Move.from_uci('f6e4'), Move.from_uci('d2d4'), Move.from_uci('e4c3'), Move.from_uci('b2c3'), Move.from_uci('c8e6'), Move.from_uci('f1d3'), Move.from_uci('f7f6'), Move.from_uci('e1g1'), Move.from_uci('f6e5'), Move.from_uci('f3e5'), Move.from_uci('c6e5'), Move.from_uci('d4e5'), Move.from_uci('d8d7'), Move.from_uci('c1e3'), Move.from_uci('d7c6'), Move.from_uci('d1d2'), Move.from_uci('e8f7'), Move.from_uci('f1e1'), Move.from_uci('f7g8'), Move.from_uci('a1d1'), Move.from_uci('e6g4'), Move.from_uci('d3e2'), Move.from_uci('g4e6'), Move.from_uci('e2f3'), Move.from_uci('a8d8'), Move.from_uci('e3a7'), Move.from_uci('c6a6'), Move.from_uci('a7d4'), Move.from_uci('a6a2'), 

# 3) Leela Chess vs. Random Agent

In [10]:
!git clone https://github.com/Zeta36/chess-alpha-zero

fatal: destination path 'chess-alpha-zero' already exists and is not an empty directory.


In [11]:
import os, sys
sys.path.append(os.path.join('./chess-alpha-zero', 'src'))

from chess_zero.agent.player_chess import ChessPlayer
from chess_zero.config import Config, PlayWithHumanConfig
from chess_zero.env.chess_env import ChessEnv

from chess_zero.agent.model_chess import ChessModel
from chess_zero.lib.model_helper import load_best_model_weight

import h5py
import tensorflow
import keras

In [12]:
#thread.daemon = False
config = Config()
PlayWithHumanConfig().update_play_config(config.play)

model = ChessModel(config)
player = ChessPlayer(config, model.get_pipes(config.play.search_threads))

In [ ]:
env = ChessEnv().reset()
action = player.action(env, False)

env

Exception in thread prediction_worker:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "./chess-alpha-zero/src/chess_zero/agent/api_chess.py", line 67, in _predict_batch_worker
    policy_ary, value_ary = self.agent_model.model.predict_on_batch(data)
AttributeError: 'NoneType' object has no attribute 'predict_on_batch'



In [14]:
print(f"bestmove {action}")

bestmove f5e6
